# Imports

In [188]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from statsmodels.formula.api import ols
import pandas as pd
import numpy as np
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import numpy as np
from sklearn import datasets
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GroupShuffleSplit
import os
import struct
import datetime
# from skfda import FDataGrid
# from skfda.preprocessing.dim_reduction import FPCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np
import pandas as pd
import struct
import matplotlib.pyplot as plt
import torch
from scipy.ndimage import gaussian_filter1d
import torch.nn as nn
import splitfolders
from tensorflow.keras.models import load_model
from scipy.interpolate import CubicSpline
from scipy.signal import find_peaks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import torch
from PIL import Image
from torchvision.transforms import ToTensor
import random
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np4
from sklearn.metrics import r2_score
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score
import torch
import torch.distributed
import pandas as pd
import numpy as np
from ABRA_35 import interpolate_and_smooth, CNN, plot_wave, calculate_and_plot_wave, plot_waves_single_frequency, arfread, get_str, calculate_hearing_threshold, all_thresholds, peak_finding
import warnings
from sklearn.preprocessing import StandardScaler,MinMaxScaler
warnings.filterwarnings('ignore')
import os
import io
import re
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [47]:
def CV_modeltesting(K, X, y, mod):
    kf = KFold(n_splits=K)

    train_fold_RMSES = []
    test_fold_RMSES = []

    for i, (train_index, test_index) in enumerate(kf.split(X)):

        print(f'Processing Fold {i+1}')
        X_train, y_train = X.iloc[train_index,:], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index,:], y.iloc[test_index]

        y_preds_train = mod.predict(X_train)
        RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))
        print(f'Train RMSE = {RMSE_train}')
        train_fold_RMSES.append(RMSE_train)

        y_preds_test = mod.predict(X_test)
        RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))
        print(f'Test RMSE = {RMSE_test}\n')
        test_fold_RMSES.append(RMSE_test)

    avg_train_RMSE = np.mean(train_fold_RMSES)
    avg_test_RMSE = np.mean(test_fold_RMSES)

    print(f'Average Train RMSE across {K} folds: {avg_train_RMSE}')
    print(f'Average Test RMSE across {K} folds: {avg_test_RMSE}')

    return avg_train_RMSE, avg_test_RMSE

In [48]:
def stratified_kfold_by_feature(X, y, model, stratify_col, n_splits=5, random_state=42):
    """
    Perform cross-validation stratified by a specific feature column,
    but exclude that column from being used as a predictor.
    
    Parameters:
    -----------
    X : DataFrame
        The feature matrix (must be a pandas DataFrame)
    y : Series or array-like
        The target variable
    model : estimator object
        The model to evaluate
    stratify_col : str
        Name of the column in X to stratify by (will be excluded from predictors)
    n_splits : int, default=5
        Number of folds
    random_state : int, default=42
        Random state for reproducibility
        
    Returns:
    --------
    train_rmse_scores : list
        RMSE scores for training sets
    test_rmse_scores : list
        RMSE scores for test sets
    """
    # Ensure X is a DataFrame
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas DataFrame to use column name for stratification")
    
    # Extract the stratification column
    stratify_values = X[stratify_col].values
    
    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Lists to store the RMSE scores for each fold
    train_rmse_scores = []
    test_rmse_scores = []
    
    # Remove the stratify column from the features
    X_for_model = X.drop(columns=[stratify_col])
    
    # Iterate through each fold
    for i, (train_index, test_index) in enumerate(skf.split(X, stratify_values)):
        print(f'Processing Fold {i+1}')
        
        # Split the data
        X_train, X_test = X_for_model.iloc[train_index], X_for_model.iloc[test_index]
        
        if isinstance(y, pd.Series):
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        else:
            y_train, y_test = y[train_index], y[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate RMSE
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        # Store the scores
        train_rmse_scores.append(train_rmse)
        test_rmse_scores.append(test_rmse)
        
        # Print fold statistics with stratification distribution
        train_strat_dist = X.iloc[train_index][stratify_col].value_counts(normalize=True).to_dict()
        test_strat_dist = X.iloc[test_index][stratify_col].value_counts(normalize=True).to_dict()
        
        print(f'Fold {i+1} - Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}')
        print(f'  Train distribution of {stratify_col}: {train_strat_dist}')
        print(f'  Test distribution of {stratify_col}: {test_strat_dist}')
    
    # Calculate and print average scores
    avg_train_rmse = np.mean(train_rmse_scores)
    avg_test_rmse = np.mean(test_rmse_scores)
    
    print(f'\nAverage Train RMSE: {avg_train_rmse:.4f}')
    print(f'Average Test RMSE: {avg_test_rmse:.4f}')
    
    return train_rmse_scores, test_rmse_scores

In [252]:
def stratified_kfold_by_feature2(X, y, model, stratify_col, n_splits=5, random_state=42):
    """
    Perform cross-validation stratified by a specific feature column,
    but exclude that column from being used as a predictor.
    
    Parameters:
    -----------
    X : DataFrame
        The feature matrix (must be a pandas DataFrame)
    y : Series or array-like
        The target variable
    model : estimator object
        The model to evaluate
    stratify_col : str
        Name of the column in X to stratify by (will be excluded from predictors)
    n_splits : int, default=5
        Number of folds
    random_state : int, default=42
        Random state for reproducibility
        
    Returns:
    --------
    train_rmse_scores : list
        RMSE scores for training sets
    test_rmse_scores : list
        RMSE scores for test sets
    """
    # Ensure X is a DataFrame
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas DataFrame to use column name for stratification")
    
    # Extract the stratification column
    stratify_values = X[stratify_col].values
    
    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Lists to store the RMSE scores for each fold
    train_rmse_scores = []
    test_rmse_scores = []
    
    # Remove the stratify column from the features
    X_for_model = X.drop(columns=[stratify_col])
    
    # Iterate through each fold
    for i, (train_index, test_index) in enumerate(skf.split(X, stratify_values)):
        print(f'Processing Fold {i+1}')
        
        # Split the data
        X_train, X_test = X_for_model.iloc[train_index], X_for_model.iloc[test_index]
        
        if isinstance(y, pd.Series):
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        else:
            y_train, y_test = y[train_index], y[test_index]
        
        # Train the model
        # model.fit(X_train, y_train)
        
        # Make predictions
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        
        # Calculate RMSE
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        # Store the scores
        train_rmse_scores.append(train_rmse)
        test_rmse_scores.append(test_rmse)
        
        # Print fold statistics with stratification distribution
        train_strat_dist = X.iloc[train_index][stratify_col].value_counts(normalize=True).to_dict()
        test_strat_dist = X.iloc[test_index][stratify_col].value_counts(normalize=True).to_dict()
        
        print(f'Fold {i+1} - Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}')
        # print(f'  Train distribution of {stratify_col}: {train_strat_dist}')
        # print(f'  Test distribution of {stratify_col}: {test_strat_dist}')
    
    # Calculate and print average scores
    avg_train_rmse = np.mean(train_rmse_scores)
    avg_test_rmse = np.mean(test_rmse_scores)
    
    print(f'\nAverage Train RMSE: {avg_train_rmse:.4f}')
    print(f'Average Test RMSE: {avg_test_rmse:.4f}')
    
    
    return train_rmse_scores, test_rmse_scores

In [254]:
def stratified_grouped_kfold_by_feature_ridge(X, y, data, group_col, stratify_col, numerical_cols, n_splits=5, 
                                             random_state=42, alpha=0.01, 
                                             formula='CrossFreqSIHCRatio ~ FreqkHz_scaled + Amplitude_scaled + FreqkHz_scaled*Amplitude_scaled'):
    """
    Perform group k-fold cross-validation with approximate stratification.
    This ensures that data from the same subject stays together while approximately maintaining the distribution of stratify_col.
    """
    # Get unique groups and their corresponding stratify values
    unique_groups = data[group_col].unique()
    group_to_stratify = {group: data[data[group_col] == group][stratify_col].iloc[0] for group in unique_groups}
    
    # Create a DataFrame with group-level info
    group_df = pd.DataFrame({
        'group': list(group_to_stratify.keys()),
        'stratify_value': list(group_to_stratify.values())
    })
    
    # Split at the group level with stratification
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    group_splits = list(skf.split(group_df, group_df['stratify_value']))
    
    RMSE_train_total = 0
    RMSE_test_total = 0
    
    for i, (train_groups_idx, test_groups_idx) in enumerate(group_splits):
        train_groups = group_df.iloc[train_groups_idx]['group'].values
        test_groups = group_df.iloc[test_groups_idx]['group'].values
        
        train_mask = data[group_col].isin(train_groups)
        test_mask = data[group_col].isin(test_groups)
        
        train_data = data[train_mask].copy()
        test_data = data[test_mask].copy()
        
        for col in numerical_cols:
            scaler = StandardScaler()
            scaler.fit(train_data[[col]])
            train_data[f'{col}_scaled'] = scaler.transform(train_data[[col]])
            test_data[f'{col}_scaled'] = scaler.transform(test_data[[col]])
        
        model = smf.ols(formula, data=train_data).fit_regularized(L1_wt=0.0, alpha=alpha)
        
        y_preds_train = model.predict(train_data)
        RMSE_train = np.sqrt(mean_squared_error(train_data['CrossFreqSIHCRatio'], y_preds_train))
        RMSE_train_total += RMSE_train
        
        y_preds_test = model.predict(test_data)
        RMSE_test = np.sqrt(mean_squared_error(test_data['CrossFreqSIHCRatio'], y_preds_test))
        RMSE_test_total += RMSE_test
        
        print(f"Fold {i+1}/{n_splits}: Train RMSE = {RMSE_train:.4f}, Test RMSE = {RMSE_test:.4f}")
        print(f"              Train groups: {len(train_groups)}, Test groups: {len(test_groups)}")
        print(f"              Train data points: {len(train_data)}, Test data points: {len(test_data)}")
        
    avg_train_RMSE = RMSE_train_total / n_splits
    avg_test_RMSE = RMSE_test_total / n_splits
    
    print(f"\nFinal Results - Avg Train RMSE = {avg_train_RMSE:.5f}, Avg Test RMSE = {avg_test_RMSE:.5f}")
    
    return avg_train_RMSE, avg_test_RMSE

In [255]:
# def stratified_kfold_by_feature_ridge(X, y, data, stratify_col, numerical_cols, n_splits=5, random_state=42, alpha=0.01, formula='CrossFreqSIHCRatio ~ FreqkHz_scaled + Amplitude_scaled + FreqkHz_scaled*Amplitude_scaled'):
#     stratify_values = X[stratify_col].values
#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     RMSE_train_total = 0
#     RMSE_test_total = 0
#     for i, (train_index, test_index) in enumerate(skf.split(X, stratify_values)):

#         train_data = data.iloc[train_index].copy()
#         test_data = data.iloc[test_index].copy()

#         scaler = StandardScaler()
#         for col in numerical_cols:
#             scaler.fit(train_data[[col]])
#             train_data[f'{col}_scaled'] = scaler.transform(train_data[[col]])
#             test_data[f'{col}_scaled'] = scaler.transform(test_data[[col]])


#         model = smf.ols(formula, data=train_data).fit_regularized(L1_wt=0.0, alpha=alpha)

#         y_preds_train = model.predict(train_data)
#         RMSE_train = np.sqrt(mean_squared_error(train_data['CrossFreqSIHCRatio'], y_preds_train))
#         RMSE_train_total += RMSE_train

#         y_preds_test = model.predict(test_data)
#         RMSE_test = np.sqrt(mean_squared_error(test_data['CrossFreqSIHCRatio'], y_preds_test))
#         RMSE_test_total += RMSE_test

#         print(f"Fold {i+1}/{n_splits}: Train RMSE = {RMSE_train:.4f}, Test RMSE = {RMSE_test:.4f}")

#     avg_train_RMSE = RMSE_train_total / n_splits
#     avg_test_RMSE = RMSE_test_total / n_splits

#     print(f"\nFinal Results - Avg Train RMSE = {avg_train_RMSE:.5f}, Avg Test RMSE = {avg_test_RMSE:.5f}")
#     # return avg_train_RMSE, avg_test_RMSE
    

# Data Processing

In [49]:
filter1 = 128
filter2 = 32
dropout1 = 0.5
dropout2 = 0.3
dropout_fc = 0.1

# Model initialization
peak_finding_model = CNN(filter1, filter2, dropout1, dropout2, dropout_fc)
model_loader = torch.load('./models/waveI_cnn.pth')
peak_finding_model.load_state_dict(model_loader)
peak_finding_model.eval()

def peak_finding(wave):
    # Prepare waveform
    waveform=interpolate_and_smooth(wave) # Added indexing per calculate and plot wave function
    # waveform_torch = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0) archived ABRA
    waveform_torch = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0).unsqueeze(0) #newer ABRA
    # print(waveform_torch)
    # Get prediction from model
    outputs = peak_finding_model(waveform_torch)
    prediction = int(round(outputs.detach().numpy()[0][0], 0))
    # prediction_test = int(round(outputs.detach().numpy()[0], 0))
    # print("Model output:", outputs, "Prediction true start:", prediction)

    # Apply Gaussian smoothing
    smoothed_waveform = gaussian_filter1d(waveform, sigma=1)

    # Find peaks and troughs
    n = 18
    t = 14
    # start_point = prediction - 9 archived ABRA
    start_point = prediction - 6 #newer ABRA
    smoothed_peaks, _ = find_peaks(smoothed_waveform[start_point:], distance=n)
    smoothed_troughs, _ = find_peaks(-smoothed_waveform, distance=t)
    sorted_indices = np.argsort(smoothed_waveform[smoothed_peaks+start_point])
    highest_smoothed_peaks = np.sort(smoothed_peaks[sorted_indices[-5:]] + start_point)
    relevant_troughs = np.array([])
    for p in range(len(highest_smoothed_peaks)):
        c = 0
        for t in smoothed_troughs:
            if t > highest_smoothed_peaks[p]:
                if p != 4:
                    try:
                        if t < highest_smoothed_peaks[p+1]:
                            relevant_troughs = np.append(relevant_troughs, int(t))
                            break
                    except IndexError:
                        pass
                else:
                    relevant_troughs = np.append(relevant_troughs, int(t))
                    break
    relevant_troughs = relevant_troughs.astype('i')
    return highest_smoothed_peaks, relevant_troughs

def extract_metadata(metadata_lines):
    # Dictionary to store extracted metadata
    metadata = {}
    
    for line in metadata_lines:
        # Extract SW FREQ
        freq_match = re.search(r'SW FREQ:\s*(\d+\.?\d*)', line)
        if freq_match:
            metadata['SW_FREQ'] = float(freq_match.group(1))
        
        # Extract LEVELS
        levels_match = re.search(r':LEVELS:\s*([^:]+)', line)
        if levels_match:
            # Split levels and convert to list of floats
            metadata['LEVELS'] = [float(level) for level in levels_match.group(1).split(';') if level]
    
    return metadata

def read_custom_tsv(file_path):
    # Read the entire file
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        content = f.read()
    
    # Split the content into metadata and data sections
    metadata_lines = []
    data_section = None
    
    # Find the ':DATA' marker
    data_start = content.find(':DATA')
    
    if data_start != -1:
        # Extract metadata (lines before ':DATA')
        metadata_lines = content[:data_start].split('\n')
        
        # Extract data section
        data_section = content[data_start:].split(':DATA')[1].strip()
    
    # Extract specific metadata
    metadata = extract_metadata(metadata_lines)
    
    # Read the data section directly
    try:
        # Use StringIO to create a file-like object from the data section
        raw_data = pd.read_csv(
            io.StringIO(data_section), 
            sep='\s+',  # Use whitespace as separator
            header=None
        )
        raw_data = raw_data.T
        # Add metadata columns to the DataFrame
        if 'SW_FREQ' in metadata:
            raw_data['Freq(kHz)'] = metadata['SW_FREQ']
            # raw_data['Freq(Hz)'] = raw_data['Freq(Hz)'].apply(lambda x: x*1000)
        
        if 'LEVELS' in metadata:
            # Repeat levels to match the number of rows
            levels_repeated = metadata['LEVELS'] * (len(raw_data) // len(metadata['LEVELS']) + 1)
            raw_data['Level(dB)'] = levels_repeated[:len(raw_data)]
        
        filtered_data = raw_data.apply(pd.to_numeric, errors='coerce').dropna()
        filtered_data.columns = filtered_data.columns.map(str)

        columns = ['Freq(kHz)'] + ['Level(dB)'] + [col for col in filtered_data.columns if col.isnumeric() == True]
        filtered_data = filtered_data[columns]
        return filtered_data
    
    except Exception as e:
        print(f"Error reading data: {e}")
        return None, metadata


def peaks_troughs_amp_final(df, freq, db, time_scale=10, multiply_y_factor=1.0, units='Microvolts'):
    db_column = 'Level(dB)'
    
    khz = df[(df['Freq(kHz)'] == freq) & (df[db_column] == db)]
    if not khz.empty:
        index = khz.index.values[0]
        final = df.loc[index, '0':].dropna()
        final = pd.to_numeric(final, errors='coerce').dropna()

        target = int(244 * (time_scale / 10))
        
        # Process the wave as in calculate_and_plot_wave
        y_values = interpolate_and_smooth(final, target)
        
        # Apply scaling factor
        y_values *= multiply_y_factor
        
        # Handle units conversion if needed
        if units == 'Nanovolts':
            y_values /= 1000
            
        # Generate normalized version for peak finding
        y_values_fpf = interpolate_and_smooth(y_values[:244])
        
        # Standardize and normalize for peak finding, exactly as in the original
        flattened_data = y_values_fpf.flatten().reshape(-1, 1)
        scaler = StandardScaler()
        standardized_data = scaler.fit_transform(flattened_data)
        min_max_scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = min_max_scaler.fit_transform(standardized_data).reshape(y_values_fpf.shape)
        y_values_fpf = interpolate_and_smooth(scaled_data[:244])
        
        # Find peaks using the normalized data
        highest_peaks, relevant_troughs = peak_finding(y_values_fpf)
        
        # Calculate amplitude on the processed but non-normalized data
        if highest_peaks.size > 0 and relevant_troughs.size > 0:
            # Following the same approach as in the display_metrics_table function
            first_peak_amplitude = y_values[highest_peaks[0]] - y_values[relevant_troughs[0]]
            return highest_peaks, relevant_troughs, first_peak_amplitude
    
    return None, None, None

In [50]:
#ORIGINAL DATA SPLIT

time_scale = 18
amp_per_freq = {'Subject': [], 'Freq(kHz) (x1)': [], 'Level(dB) (x2)': [], 'Amplitude (x3)':[]}
start_path = '/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/abr_data/WPZ Electrophysiology'
for subject in os.listdir(start_path):
    # print("Subject:",subject)
    for fq in os.listdir(os.path.join(start_path,subject)):
        # print(fq)
        if fq.startswith('ABR') and fq.endswith('.tsv'):
            path = os.path.join(start_path,subject,fq)
            data_df = read_custom_tsv(path)
            # print(data_df)
            freqs = data_df['Freq(kHz)'].unique().tolist()
            levels = data_df['Level(dB)'].unique().tolist()
            for freq in freqs:
                for lvl in levels:
                    # print("Frequency=",freq, "Level=", lvl)
                    _, _, amp = peaks_troughs_amp_final(df=data_df, freq=freq, db=lvl, time_scale=time_scale)
                    # print(f'Amplitude: {amp}\n')
                    amp_per_freq['Subject'].append(subject)
                    amp_per_freq['Freq(kHz) (x1)'].append(freq)
                    amp_per_freq['Level(dB) (x2)'].append(lvl)
                    amp_per_freq['Amplitude (x3)'].append(amp)
        else:
            pass

amp_df_full = pd.DataFrame(data=amp_per_freq)

raw_synapse_counts = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Ribbon and Synapse Counts.xlsx')
raw_synapse_counts = raw_synapse_counts.mask(lambda x: x.isnull()).dropna()
raw_synapse_counts['Synapses to IHC (y1)'] = raw_synapse_counts.iloc[:,6]
raw_synapse_counts['vx (x4)'] = raw_synapse_counts['vx']
raw_synapse_counts.drop(columns=['vx'], inplace=True)
raw_synapse_counts.rename(columns={'Freq':'Freq(kHz) (x1)'}, inplace=True)
# raw_synapse_counts['Freq(Hz) (x1)'] = raw_synapse_counts['Freq(Hz) (x1)'].apply(lambda x: x*1000) # PUTTING BACK
raw_synapse_counts.rename(columns={'Case':'Subject', 'IHCs' : 'IHCs (y2)'}, inplace=True)

paired = amp_df_full.join(raw_synapse_counts.set_index(['Subject', 'Freq(kHz) (x1)']), on=['Subject', 'Freq(kHz) (x1)'])
slice = paired[paired['Subject']=='WPZ174'][['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'IHCs (y2)']]
final = paired[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'IHCs (y2)']]
final_clean = final.dropna()

# adding in the strain feature
strains = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Mouse groups.xlsx')
final_clean_strained = final_clean.join(strains.set_index('ID#'), on='Subject')
final_clean_strained['Strain'] = final_clean_strained['Strain'].str.strip()
final_clean_strained = final_clean_strained.rename(columns={'Strain': 'Strain (x5)'})
final_clean_strained = final_clean_strained.dropna()
final_clean_strained_foundation = final_clean_strained.copy()
final_clean_strained = final_clean_strained[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)', 'Strain (x5)', 'Synapses to IHC (y1)', 'Group']]
np.unique(final_clean_strained['Group'])

final_clean_70 = final_clean[final_clean['Level(dB) (x2)'] >= 70.0]
final_clean_strained_70 = final_clean_strained[final_clean_strained['Level(dB) (x2)'] >= 70.0]
# np.unique(final_clean['Level(dB) (x2)']) max level is 80 db
len(final_clean), len(final_clean_70) # 10000 less data points!!!

final_clean_strained_grouped = final_clean_strained.copy()
final_clean_strained_grouped['Group - dB'] = final_clean_strained_grouped['Group'].apply(lambda x: x.split(' ')[0] if x.split(' ')[0].endswith('dB') else 'Control')
final_clean_strained_grouped['Group - Time Elapsed'] = final_clean_strained_grouped['Group'].apply(lambda x: x.split(' ')[1] if x.split(' ')[1].endswith(('h', 'wks', 'w')) else x.split(' ')[0])
final_clean_strained_grouped.head()

final_clean_strained_grouped_pos = final_clean_strained_grouped.copy()
final_clean_strained_grouped_pos['Amplitude (x3)'] = final_clean_strained_grouped['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

len(final_clean_strained_grouped_pos[final_clean_strained_grouped_pos['Amplitude (x3)'] < 0])

final_clean_strained_grouped_pos['Amplitude (x3)'] = final_clean_strained_grouped['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

# final_clean_strained_grouped_pos[(final_clean_strained_grouped_pos['Subject'] == 'WPZ66') & (final_clean_strained_grouped_pos['Amplitude (x3)'] ==0.055901451434921576)
final_clean_strained_grouped_pos_cleangroup = final_clean_strained_grouped_pos.copy()
final_clean_strained_grouped_pos_cleangroup['Group'] = final_clean_strained_grouped_pos_cleangroup['Group'].apply(lambda x: x.strip())
np.unique(final_clean_strained_grouped_pos_cleangroup['Group'])

final_clean_strained_grouped_pos_cleangroup.head()
final_clean_strained_grouped_pos_cleangroup_vs = final_clean_strained_grouped_pos_cleangroup.copy()
final_clean_strained_grouped_pos_cleangroup_vs['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs['Group - dB']
# final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed']
final_clean_strained_grouped_pos_cleangroup_vs = final_clean_strained_grouped_pos_cleangroup_vs[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)',
       'vx (x4)', 'Strain (x5)','Group - dB (x6)', 'Group - Time Elapsed', 'Group','Synapses to IHC (y1)']]

def split_on_number(input_string):
    return re.findall(r"[A-Za-z]+|\d+", input_string)

hrs_week = 24*7

final_clean_strained_grouped_pos_cleangroup_vs_timed = final_clean_strained_grouped_pos_cleangroup_vs.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda x: '0dB' if x == 'Control' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Split'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Split'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Magn.'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Split'].apply(lambda x: x[1])
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Time Elapsed - Unit'].apply(lambda x: "wks" if x == 'w' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Hours Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed.apply(lambda row: row['Group - Time Elapsed - Magn.']* hrs_week if row['Group - Time Elapsed - Unit'] == 'wks' else row['Group - Time Elapsed - Magn.'], axis = 1)

# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Days Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Hours Elapsed (x7)'].apply(lambda x: x/24)
# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda )

In [51]:
# NEW DATA SPLIT

time_scale = 18
amp_per_freq = {'Subject': [], 'Freq(kHz) (x1)': [], 'Level(dB) (x2)': [], 'Amplitude (x3)':[]}
start_path = '/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/abr_data/WPZ Electrophysiology'
for subject in os.listdir(start_path):
    # print("Subject:",subject)
    for fq in os.listdir(os.path.join(start_path,subject)):
        # print(fq)
        if fq.startswith('ABR') and fq.endswith('.tsv'):
            path = os.path.join(start_path,subject,fq)
            data_df = read_custom_tsv(path)
            # print(data_df)
            freqs = data_df['Freq(kHz)'].unique().tolist()
            levels = data_df['Level(dB)'].unique().tolist()
            for freq in freqs:
                for lvl in levels:
                    # print("Frequency=",freq, "Level=", lvl)
                    _, _, amp = peaks_troughs_amp_final(df=data_df, freq=freq, db=lvl, time_scale=time_scale)
                    # print(f'Amplitude: {amp}\n')
                    amp_per_freq['Subject'].append(subject)
                    amp_per_freq['Freq(kHz) (x1)'].append(freq)
                    amp_per_freq['Level(dB) (x2)'].append(lvl)
                    amp_per_freq['Amplitude (x3)'].append(amp)
        else:
            pass

amp_df_full = pd.DataFrame(data=amp_per_freq)

raw_synapse_counts = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Ribbon and Synapse Counts.xlsx')
raw_synapse_counts = raw_synapse_counts.mask(lambda x: x.isnull()).dropna()
raw_synapse_counts['Synapses to IHC (y1)'] = raw_synapse_counts.iloc[:,6]
raw_synapse_counts['vx (x4)'] = raw_synapse_counts['vx']
raw_synapse_counts.drop(columns=['vx'], inplace=True)
raw_synapse_counts.rename(columns={'Freq':'Freq(kHz) (x1)'}, inplace=True)
# raw_synapse_counts['Freq(Hz) (x1)'] = raw_synapse_counts['Freq(Hz) (x1)'].apply(lambda x: x*1000) # PUTTING BACK
raw_synapse_counts.rename(columns={'Case':'Subject'}, inplace=True)

paired2 = amp_df_full.join(raw_synapse_counts.set_index(['Subject', 'Freq(kHz) (x1)']), on=['Subject', 'Freq(kHz) (x1)'])
# lilslice = paired[paired['Subject']=='WPZ174'][['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'IHCs']]
final2 = paired2[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)','Synapses to IHC (y1)', 'Synapses', 'IHCs']]
final_clean2 = final2.dropna()

# adding in the strain feature
strains = pd.read_excel('/Users/leahashebir/Downloads/Manor_Practicum/liberman_data/WPZ Mouse groups.xlsx')
final_clean_strained2 = final_clean2.join(strains.set_index('ID#'), on='Subject')
final_clean_strained2['Strain'] = final_clean_strained2['Strain'].str.strip()
final_clean_strained2 = final_clean_strained2.rename(columns={'Strain': 'Strain (x5)'})
final_clean_strained2 = final_clean_strained2.dropna()
final_clean_strained2 = final_clean_strained2[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)', 'vx (x4)', 'Strain (x5)', 'Synapses to IHC (y1)', 'Group', 'Synapses', 'IHCs']]
# np.unique(final_clean_strained2['Group'])

# final_clean_70 = final_clean[final_clean['Level(dB) (x2)'] >= 70.0]
# final_clean_strained_70 = final_clean_strained[final_clean_strained['Level(dB) (x2)'] >= 70.0]
# # np.unique(final_clean['Level(dB) (x2)']) max level is 80 db
# len(final_clean), len(final_clean_70) # 10000 less data points!!!

final_clean_strained_grouped2 = final_clean_strained2.copy()
final_clean_strained_grouped2['Group - dB'] = final_clean_strained_grouped2['Group'].apply(lambda x: x.split(' ')[0] if x.split(' ')[0].endswith('dB') else 'Control')
final_clean_strained_grouped2['Group - Time Elapsed'] = final_clean_strained_grouped2['Group'].apply(lambda x: x.split(' ')[1] if x.split(' ')[1].endswith(('h', 'wks', 'w')) else x.split(' ')[0])
final_clean_strained_grouped2.head()

final_clean_strained_grouped_pos2 = final_clean_strained_grouped2.copy()
final_clean_strained_grouped_pos2['Amplitude (x3)'] = final_clean_strained_grouped2['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

len(final_clean_strained_grouped_pos2[final_clean_strained_grouped_pos2['Amplitude (x3)'] < 0])

final_clean_strained_grouped_pos2['Amplitude (x3)'] = final_clean_strained_grouped2['Amplitude (x3)'].apply(lambda x: 0 if x < 0 else x)

# final_clean_strained_grouped_pos[(final_clean_strained_grouped_pos['Subject'] == 'WPZ66') & (final_clean_strained_grouped_pos['Amplitude (x3)'] ==0.055901451434921576)
final_clean_strained_grouped_pos_cleangroup2 = final_clean_strained_grouped_pos2.copy()
final_clean_strained_grouped_pos_cleangroup2['Group'] = final_clean_strained_grouped_pos_cleangroup2['Group'].apply(lambda x: x.strip())
np.unique(final_clean_strained_grouped_pos_cleangroup2['Group'])

final_clean_strained_grouped_pos_cleangroup2.head()
final_clean_strained_grouped_pos_cleangroup_vs2 = final_clean_strained_grouped_pos_cleangroup2.copy()
final_clean_strained_grouped_pos_cleangroup_vs2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs2['Group - dB']
# final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs['Group - Time Elapsed']
final_clean_strained_grouped_pos_cleangroup_vs2 = final_clean_strained_grouped_pos_cleangroup_vs2[['Subject', 'Freq(kHz) (x1)', 'Level(dB) (x2)', 'Amplitude (x3)',
       'vx (x4)', 'Strain (x5)','Group - dB (x6)', 'Group - Time Elapsed', 'Group','Synapses to IHC (y1)', 'Synapses', 'IHCs']]

def split_on_number(input_string):
    return re.findall(r"[A-Za-z]+|\d+", input_string)

hrs_week = 24*7

final_clean_strained_grouped_pos_cleangroup_vs_timed2 = final_clean_strained_grouped_pos_cleangroup_vs2.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(lambda x: '0dB' if x == 'Control' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - dB (x6)'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Split'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed'].apply(split_on_number)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Split'].apply(lambda x: x[0])
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Magn.'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Magn.'].apply(lambda x: int(x.strip()))
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Split'].apply(lambda x: x[1])
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Unit'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Time Elapsed - Unit'].apply(lambda x: "wks" if x == 'w' else x)
final_clean_strained_grouped_pos_cleangroup_vs_timed2['Group - Hours Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2.apply(lambda row: row['Group - Time Elapsed - Magn.']* hrs_week if row['Group - Time Elapsed - Unit'] == 'wks' else row['Group - Time Elapsed - Magn.'], axis = 1)
final_clean_strained_grouped_pos_cleangroup_vs_timed2
# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Days Elapsed (x7)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - Hours Elapsed (x7)'].apply(lambda x: x/24)
# final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed['Group - dB (x6)'].apply(lambda )

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Strain (x5),Group - dB (x6),Group - Time Elapsed,Group,Synapses to IHC (y1),Synapses,IHCs,Group - Time Elapsed - Split,Group - Time Elapsed - Magn.,Group - Time Elapsed - Unit,Group - Hours Elapsed (x7)
0,WPZ145,45.2,70.0,0.033579,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344
0,WPZ145,45.2,70.0,0.033579,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344
1,WPZ145,45.2,75.0,0.034262,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344
1,WPZ145,45.2,75.0,0.034262,v2,C57B6,98,8wks,98dB 8wks post,10.888889,98.0,9,"[8, wks]",8,wks,1344
2,WPZ145,45.2,80.0,0.154224,v1,C57B6,98,8wks,98dB 8wks post,8.750000,77.0,8.8,"[8, wks]",8,wks,1344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,WPZ101,32.0,60.0,1.634279,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344
7329,WPZ101,32.0,70.0,1.769193,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344
7329,WPZ101,32.0,70.0,1.769193,v2,C57B6,0,8wks,8wks ctrl,15.463918,150.0,9.7,"[8, wks]",8,wks,1344
7330,WPZ101,32.0,80.0,1.023154,v1,C57B6,0,8wks,8wks ctrl,16.923077,154.0,9.1,"[8, wks]",8,wks,1344


In [52]:
final_clean2

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Synapses to IHC (y1),Synapses,IHCs
0,WPZ145,45.2,70.0,0.033579,v1,8.750000,77.0,8.8
0,WPZ145,45.2,70.0,0.033579,v2,10.888889,98.0,9
1,WPZ145,45.2,75.0,0.034262,v1,8.750000,77.0,8.8
1,WPZ145,45.2,75.0,0.034262,v2,10.888889,98.0,9
2,WPZ145,45.2,80.0,0.154224,v1,8.750000,77.0,8.8
...,...,...,...,...,...,...,...,...
7328,WPZ101,32.0,60.0,1.634279,v2,15.463918,150.0,9.7
7329,WPZ101,32.0,70.0,1.769193,v1,16.923077,154.0,9.1
7329,WPZ101,32.0,70.0,1.769193,v2,15.463918,150.0,9.7
7330,WPZ101,32.0,80.0,1.023154,v1,16.923077,154.0,9.1


# Vanilla Lin Reg Model

In [53]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed[['Freq(kHz) (x1)', 'Level(dB) (x2)','Amplitude (x3)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed['Synapses to IHC (y1)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42)

In [54]:
model_vanilla = LinearRegression().fit(X_train,y_train)

y_preds_train = model_vanilla.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_vanilla.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.8962291261784285, 2.895859167040614)

In [55]:
# Cross validation
K=5
kf = KFold(n_splits=K)

train_fold_RMSES = []
test_fold_RMSES = []

for i, (train_index, test_index) in enumerate(kf.split(X)):

    print(f'Processing Fold {i+1}')
    X_train, y_train = X.iloc[train_index,:], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index,:], y.iloc[test_index]

    y_preds_train = model_vanilla.predict(X_train)
    RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))
    print(f'Train RMSE = {RMSE_train}')
    train_fold_RMSES.append(RMSE_train)

    y_preds_test = model_vanilla.predict(X_test)
    RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))
    print(f'Test RMSE = {RMSE_test}\n')
    test_fold_RMSES.append(RMSE_test)

avg_train_RMSE = np.mean(train_fold_RMSES)
avg_test_RMSE = np.mean(test_fold_RMSES)

print(f'Average Train RMSE across {K} folds: {avg_train_RMSE}')
print(f'Average Test RMSE across {K} folds: {avg_test_RMSE}')

Processing Fold 1
Train RMSE = 2.913936454557555
Test RMSE = 2.8239328735809175

Processing Fold 2
Train RMSE = 2.98472400080733
Test RMSE = 2.5109567657958833

Processing Fold 3
Train RMSE = 2.8023395289110073
Test RMSE = 3.2444697246817285

Processing Fold 4
Train RMSE = 2.9618158381505606
Test RMSE = 2.617029707707716

Processing Fold 5
Train RMSE = 2.8131074445144497
Test RMSE = 3.2069717077308724

Average Train RMSE across 5 folds: 2.8951846533881804
Average Test RMSE across 5 folds: 2.8806721558994233


## Sign Tests

In [56]:
final_clean_strained_grouped_pos_cleangroup_vs_timed_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed_ols.rename(columns = {'Freq(kHz) (x1)' : 'FreqkHz', 'Level(dB) (x2)': 'LeveldB', 'Amplitude (x3)' : 'Amplitude', 'Synapses to IHC (y1)' : 'Synapses_to_IHC'})
final_clean_strained_grouped_pos_cleangroup_vs_timed_ols.columns

Index(['Subject', 'FreqkHz', 'LeveldB', 'Amplitude', 'vx (x4)', 'Strain (x5)',
       'Group - dB (x6)', 'Group - Time Elapsed', 'Group', 'Synapses_to_IHC',
       'Group - Time Elapsed - Split', 'Group - Time Elapsed - Magn.',
       'Group - Time Elapsed - Unit', 'Group - Hours Elapsed (x7)'],
      dtype='object')

In [57]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

# Full model
full_model = ols('Synapses_to_IHC ~ FreqkHz + LeveldB + Amplitude', 
                data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("Full Model Summary:")
print(full_model.summary())

# Partial F-test for Amplitude
reduced_model1 = ols('Synapses_to_IHC ~ FreqkHz + LeveldB', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for Amplitude:")
print(anova_lm(reduced_model1, full_model))

# Partial F-test for FreqkHz
reduced_model2 = ols('Synapses_to_IHC ~ LeveldB + Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for FreqkHz:")
print(anova_lm(reduced_model2, full_model))

# Partial F-test for LeveldB
reduced_model3 = ols('Synapses_to_IHC ~ FreqkHz + Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for LeveldB:")
print(anova_lm(reduced_model3, full_model))

# Partial F-test for FreqkHz and LeveldB jointly
reduced_model4 = ols('Synapses_to_IHC ~ Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed_ols).fit()
print("\nPartial F-test for FreqkHz and LeveldB together:")
print(anova_lm(reduced_model4, full_model))

Full Model Summary:
                            OLS Regression Results                            
Dep. Variable:        Synapses_to_IHC   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     389.0
Date:                Tue, 01 Apr 2025   Prob (F-statistic):          2.75e-241
Time:                        21:25:51   Log-Likelihood:                -30252.
No. Observations:               12187   AIC:                         6.051e+04
Df Residuals:                   12183   BIC:                         6.054e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.9318      0.076

# Stratified CV

In [58]:
np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed['Strain (x5)'])

array(['C57B6', 'CBA/CaJ'], dtype=object)

In [59]:
final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded = final_clean_strained_grouped_pos_cleangroup_vs_timed.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'].apply(lambda x: 0 if x == 'C57B6' else 1)

np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'])

array([0, 1])

In [60]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded[['Freq(kHz) (x1)', 'Level(dB) (x2)','Amplitude (x3)', 'Strain (x5)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Synapses to IHC (y1)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42, stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed_encoded['Strain (x5)'])


In [61]:
model_strain_strat = LinearRegression().fit(X_train,y_train)

y_preds_train = model_strain_strat.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_strain_strat.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

(2.8396751633699444, 2.8321396400722816)

In [62]:
train_rmses, test_rmses = stratified_kfold_by_feature(X, y, model_strain_strat, stratify_col='Strain (x5)', n_splits=5)

Processing Fold 1
Fold 1 - Train RMSE: 2.9084, Test RMSE: 2.8473
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 2
Fold 2 - Train RMSE: 2.9023, Test RMSE: 2.8721
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 3
Fold 3 - Train RMSE: 2.8971, Test RMSE: 2.8928
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 4
Fold 4 - Train RMSE: 2.8624, Test RMSE: 3.0302
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 5
Fold 5 - Train RMSE: 2.9084, Test RMSE: 2.8462
  Train d

# **Predicting on the Average between Viewing Fields**

## Data Processing

In [74]:
freqs = np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed2['Freq(kHz) (x1)'])
subs = np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed2['Subject'])
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx = final_clean_strained_grouped_pos_cleangroup_vs_timed2.copy()
for freq in freqs:
    for sub in subs:
        mask = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == freq) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == sub)] # global for updates
        if len(mask) > 0:

            mask1 = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == freq) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == sub) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['vx (x4)'] == 'v1')]
            mask2 = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == freq) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == sub) & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['vx (x4)'] == 'v2')]

            if not mask1.empty and not mask2.empty:
                mask1 = mask1.reset_index().iloc[0,:]
                mask2 = mask2.reset_index().iloc[0,:]

                total_syns = float(mask1['Synapses'] + mask2['Synapses'])
                total_ihcs = float(mask1['IHCs'] + mask2['IHCs'])
                ratio = total_syns / total_ihcs
                # print(total_syns, total_ihcs)
                # if total_syns == 0.0 or total_ihcs == 0.0:
                #     print(sub, freq)
                mask_index = mask.index
                final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx.loc[mask_index,'CrossFreq_SIHCRatio (y2)'] = ratio
            elif mask1.empty:
                mask2 = mask2.reset_index().iloc[0,:]
                total_syns = float(mask2['Synapses'])
                total_ihcs = float(mask2['IHCs'])
                ratio = total_syns / total_ihcs
                # print(total_syns, total_ihcs)
                # if total_syns == 0.0 or total_ihcs == 0.0:
                #     print(sub, freq)
                mask_index = mask.index
                final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx.loc[mask_index,'CrossFreq_SIHCRatio (y2)'] = ratio
            elif mask2.empty:
                mask1 = mask1.reset_index().iloc[0,:]
                total_syns = float(mask1['Synapses'])
                total_ihcs = float(mask1['IHCs'])
                ratio = total_syns / total_ihcs
                # print(total_syns, total_ihcs)
                # if total_syns == 0.0 or total_ihcs == 0.0:
                #     print(sub, freq)
                mask_index = mask.index
                final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx.loc[mask_index,'CrossFreq_SIHCRatio (y2)'] = ratio

In [78]:
# WPZ92	32.0, WPZ157	16.0
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx[(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Subject'] == 'WPZ92') & (final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx['Freq(kHz) (x1)'] == 32.0)]

,Subject,Freq(kHz) (x1),Level(dB) (x2),Amplitude (x3),vx (x4),Strain (x5),Group - dB (x6),Group - Time Elapsed,Group,Synapses to IHC (y1),Synapses,IHCs,Group - Time Elapsed - Split,Group - Time Elapsed - Magn.,Group - Time Elapsed - Unit,Group - Hours Elapsed (x7),CrossFreq_SIHCRatio (y2)
2834,WPZ92,32.0,10.0,0.106536,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2835,WPZ92,32.0,15.0,0.069769,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2836,WPZ92,32.0,20.0,0.161671,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2837,WPZ92,32.0,25.0,0.148119,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2838,WPZ92,32.0,30.0,0.209200,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2839,WPZ92,32.0,35.0,0.235307,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2840,WPZ92,32.0,40.0,0.376942,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2841,WPZ92,32.0,45.0,0.503456,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2842,WPZ92,32.0,50.0,0.509231,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0
2843,WPZ92,32.0,60.0,0.724831,v2,C57B6,90,24h,90dB 24h post,16.0,160.0,10,"[24, h]",24,h,24,16.0


## Starting Stratification by Group/Strain...

In [173]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Strain (x5)'] = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Strain (x5)'].apply(lambda x: 0 if x == 'C57B6' else 1)

encoder = LabelEncoder()
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Group'] = encoder.fit_transform(pd.DataFrame(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Group']))

np.unique(final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Group'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [176]:
X = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded[['Freq(kHz) (x1)', 'Amplitude (x3)', 'Strain (x5)']]
y = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['CrossFreq_SIHCRatio (y2)']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2, random_state=42, stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Strain (x5)'])

model_avg_strat = LinearRegression().fit(X_train,y_train)

y_preds_train = model_avg_strat.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_avg_strat.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

stratified_kfold_by_feature(X, y, model_avg_strat, 'Strain (x5)', n_splits=5, random_state=42)

Processing Fold 1
Fold 1 - Train RMSE: 2.8761, Test RMSE: 2.8129
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 2
Fold 2 - Train RMSE: 2.8743, Test RMSE: 2.8196
  Train distribution of Strain (x5): {0: 0.8502410503641399, 1: 0.1497589496358601}
  Test distribution of Strain (x5): {0: 0.850287120590648, 1: 0.14971287940935193}
Processing Fold 3
Fold 3 - Train RMSE: 2.8671, Test RMSE: 2.8489
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 4
Fold 4 - Train RMSE: 2.8186, Test RMSE: 3.0378
  Train distribution of Strain (x5): {0: 0.8502564102564103, 1: 0.14974358974358976}
  Test distribution of Strain (x5): {0: 0.850225687320476, 1: 0.149774312679524}
Processing Fold 5
Fold 5 - Train RMSE: 2.8792, Test RMSE: 2.7988
  Train d

([2.876107116157409,
  2.8742805135151,
  2.8670516441149805,
  2.8185788663873,
  2.879227826650186],
 [2.812929861567098,
  2.8195807829938655,
  2.8488572060885757,
  3.037844671442923,
  2.7988193972485713])

In [177]:
X_g = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded[['Freq(kHz) (x1)', 'Amplitude (x3)', 'Group']]
y_g = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['CrossFreq_SIHCRatio (y2)']

X_train, X_test, y_train, y_test = train_test_split(X_g, y_g, shuffle = True, test_size=0.2, random_state=42, stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded['Group'])

model_avg_strat_g = LinearRegression().fit(X_train,y_train)

y_preds_train = model_avg_strat_g.predict(X_train)
RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

y_preds_test = model_avg_strat_g.predict(X_test)
RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

RMSE_train, RMSE_test

stratified_kfold_by_feature(X_g, y_g, model_avg_strat_g, 'Group', n_splits=5, random_state=42)

Processing Fold 1
Fold 1 - Train RMSE: 2.8800, Test RMSE: 2.7958
  Train distribution of Group: {2: 0.14062980818545492, 1: 0.12667965945225151, 13: 0.09754846650938558, 6: 0.08667555646732998, 3: 0.08503436249871782, 9: 0.062057646938147505, 12: 0.05826238588573187, 7: 0.05744178890142579, 10: 0.056928915786234484, 11: 0.053031080110780594, 4: 0.053031080110780594, 8: 0.05221048312647451, 5: 0.050466714534824084, 0: 0.020002051492460766}
  Test distribution of Group: {2: 0.14068908941755537, 1: 0.12674323215750616, 13: 0.09762100082034454, 6: 0.08654634946677604, 3: 0.08490566037735849, 9: 0.061936013125512716, 12: 0.05824446267432322, 7: 0.05742411812961444, 10: 0.05701394585726005, 11: 0.05291222313371616, 4: 0.05291222313371616, 8: 0.052091878589007386, 5: 0.05086136177194422, 0: 0.020098441345365054}
Processing Fold 2
Fold 2 - Train RMSE: 2.8568, Test RMSE: 2.8891
  Train distribution of Group: {2: 0.14062980818545492, 1: 0.12667965945225151, 13: 0.09754846650938558, 6: 0.08667555

([2.879966966657405,
  2.85684376676087,
  2.8559312803125674,
  2.8578221049862336,
  2.865830441249474],
 [2.7958146695959796,
  2.8891188347463856,
  2.8930318721133785,
  2.8852682674421444,
  2.853474340888855])

## Significance Tests

In [181]:
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded.copy()
final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols.rename(columns = {'Freq(kHz) (x1)' : 'FreqkHz', 'Level(dB) (x2)': 'LeveldB', 'Amplitude (x3)' : 'Amplitude', 'CrossFreq_SIHCRatio (y2)' : 'CrossFreqSIHCRatio'})

In [182]:
model = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
FreqkHz,812.289031,1.0,106.721786,6.490847e-25
LeveldB,6049.797561,1.0,794.846634,1.834454e-169
Amplitude,5342.640872,1.0,701.937556,1.977891e-150
FreqkHz:Amplitude,997.417925,1.0,131.044762,3.453416e-30
Residual,92720.571153,12182.0,NaN,NaN


In [187]:
full_model = ols('CrossFreqSIHCRatio ~ FreqkHz + Amplitude + LeveldB', 
                data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print("Full Model Summary:")
print(full_model.summary())

# Partial F-test for Amplitude
# reduced_model1 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for Amplitude:")
# print(anova_lm(reduced_model1, full_model))

# # Partial F-test for FreqkHz
# reduced_model2 = ols('CrossFreqSIHCRatio ~ LeveldB + Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz:")
# print(anova_lm(reduced_model2, full_model))

# # Partial F-test for LeveldB
# reduced_model3 = ols('CrossFreqSIHCRatio ~ FreqkHz + Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for LeveldB:")
# print(anova_lm(reduced_model3, full_model))

# # Partial F-test for FreqkHz and LeveldB jointly
# reduced_model4 = ols('CrossFreqSIHCRatio ~ Amplitude', 
#                     data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# print("\nPartial F-test for FreqkHz and LeveldB together:")
# print(anova_lm(reduced_model4, full_model))

# Testing interactions 

#  + FreqkHz*LeveldB + LeveldB*Amplitude + FreqkHz*Amplitude + FreqkHz*LeveldB*Amplitude

reduced_model5 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*LeveldB', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print("\nPartial F-test for FreqkHz*LeveldB:")
print(anova_lm(full_model, reduced_model5))

# Partial F-test for FreqkHz
reduced_model6 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print("\nPartial F-test for FreqkHz*Amplitude:")
print(anova_lm(full_model, reduced_model6))

# Partial F-test for LeveldB
reduced_model7 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + LeveldB*Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print("\nPartial F-test for LeveldB*Amplitude:")
print(anova_lm(full_model, reduced_model7))

# # Partial F-test for FreqkHz and LeveldB jointly
reduced_model8 = ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*LeveldB + FreqkHz*Amplitude + LeveldB*Amplitude + FreqkHz*Amplitude', 
                    data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print("\nPartial F-test for ALL together:")
print(anova_lm(full_model, reduced_model8))

Full Model Summary:
                            OLS Regression Results                            
Dep. Variable:     CrossFreqSIHCRatio   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.094
Method:                 Least Squares   F-statistic:                     422.5
Date:                Tue, 01 Apr 2025   Prob (F-statistic):          4.18e-261
Time:                        23:02:43   Log-Likelihood:                -29723.
No. Observations:               12187   AIC:                         5.945e+04
Df Residuals:                   12183   BIC:                         5.948e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.9107      0.072

## With freq*amp

In [193]:
X_g = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz', 'LeveldB','Amplitude', 'Group']]
y_g = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']

model_final = smf.ols('CrossFreqSIHCRatio ~ FreqkHz + Amplitude + FreqkHz*Amplitude', data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# stratified_kfold_by_feature(X, y, model_int, 'Strain (x5)', n_splits=5, random_state=42)
# CV_modeltesting(5, X, y, model_int)

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2)

# y_preds_train = model_final.predict(X_train)
# RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

# y_preds_test = model_final.predict(X_test)
# RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

# RMSE_train, RMSE_test

stratified_kfold_by_feature2(X_g, y_g, model_final, 'Group')

Processing Fold 1
Fold 1 - Train RMSE: 2.8634, Test RMSE: 2.7797
Processing Fold 2
Fold 2 - Train RMSE: 2.8400, Test RMSE: 2.8739
Processing Fold 3
Fold 3 - Train RMSE: 2.8384, Test RMSE: 2.8805
Processing Fold 4
Fold 4 - Train RMSE: 2.8410, Test RMSE: 2.8703
Processing Fold 5
Fold 5 - Train RMSE: 2.8514, Test RMSE: 2.8285

Average Train RMSE: 2.8468
Average Test RMSE: 2.8466


([2.8633894674703457,
  2.8400406735756554,
  2.8383821283531283,
  2.8409508143625306,
  2.8514207318421323],
 [2.779740745751265,
  2.873928949589066,
  2.8804906660813816,
  2.8703415846632523,
  2.8285008855825273])

In [194]:
X_s = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz', 'LeveldB','Amplitude', 'Strain (x5)']]
y_s = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']

model_final = smf.ols('CrossFreqSIHCRatio ~ FreqkHz + Amplitude + FreqkHz*Amplitude', data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
# stratified_kfold_by_feature(X, y, model_int, 'Strain (x5)', n_splits=5, random_state=42)
# CV_modeltesting(5, X, y, model_int)

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.2)

# y_preds_train = model_final.predict(X_train)
# RMSE_train = np.sqrt(np.mean((y_train - y_preds_train)**2))

# y_preds_test = model_final.predict(X_test)
# RMSE_test = np.sqrt(np.mean((y_test - y_preds_test)**2))

# RMSE_train, RMSE_test

stratified_kfold_by_feature2(X_s, y_s, model_final, 'Strain (x5)')

Processing Fold 1
Fold 1 - Train RMSE: 2.8591, Test RMSE: 2.7973
Processing Fold 2
Fold 2 - Train RMSE: 2.8561, Test RMSE: 2.8094
Processing Fold 3
Fold 3 - Train RMSE: 2.8519, Test RMSE: 2.8266
Processing Fold 4
Fold 4 - Train RMSE: 2.8038, Test RMSE: 3.0128
Processing Fold 5
Fold 5 - Train RMSE: 2.8629, Test RMSE: 2.7818

Average Train RMSE: 2.8468
Average Test RMSE: 2.8456


([2.8591103256468577,
  2.856136125559016,
  2.851884276324155,
  2.8038325598085154,
  2.862884143136819],
 [2.797298345590854,
  2.809421695573092,
  2.8266305290599685,
  3.0128277066905222,
  2.7817875174949043])

In [197]:
stratified_kfold_by_feature2(X, y, model_final, 'Strain (x5)')

Processing Fold 1
Fold 1 - Train RMSE: 2.8591, Test RMSE: 2.7973
Processing Fold 2
Fold 2 - Train RMSE: 2.8561, Test RMSE: 2.8094
Processing Fold 3
Fold 3 - Train RMSE: 2.8519, Test RMSE: 2.8266
Processing Fold 4
Fold 4 - Train RMSE: 2.8038, Test RMSE: 3.0128
Processing Fold 5
Fold 5 - Train RMSE: 2.8629, Test RMSE: 2.7818

Average Train RMSE: 2.8468
Average Test RMSE: 2.8456


([2.8591103256468577,
  2.856136125559016,
  2.851884276324155,
  2.8038325598085154,
  2.862884143136819],
 [2.797298345590854,
  2.809421695573092,
  2.8266305290599685,
  3.0128277066905222,
  2.7817875174949043])

## Ridge - Final Model Eval

In [216]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler

numerical_cols = ['FreqkHz','Amplitude']

X_s = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz','Amplitude', 'Strain (x5)']]
y_s = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']

train_indices, test_indices = train_test_split(
    np.arange(len(X_s)), 
    shuffle=True, 
    test_size=0.2, 
    stratify=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['Strain (x5)'],
    random_state=42
)

train_data = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols.iloc[train_indices].copy()
test_data = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols.iloc[test_indices].copy()

scaler = StandardScaler()
scaler.fit(train_data[numerical_cols])

for col in numerical_cols:
    scaler.fit(train_data[[col]])
    train_data[f'{col}_scaled'] = scaler.transform(train_data[[col]])
    test_data[f'{col}_scaled'] = scaler.transform(test_data[[col]])

alphas = [0.001, 0.01, 0.1, 1.0]
ridge_results = []

for alpha in alphas:
    model_ridge = smf.ols('CrossFreqSIHCRatio ~ FreqkHz_scaled + Amplitude_scaled + FreqkHz_scaled*Amplitude_scaled', 
                         data=train_data).fit_regularized(L1_wt=0.0, alpha=alpha)
    

    y_preds_train = model_ridge.predict(train_data)
    RMSE_train = np.sqrt(mean_squared_error(train_data['CrossFreqSIHCRatio'], y_preds_train))
    
    y_preds_test = model_ridge.predict(test_data)
    RMSE_test = np.sqrt(mean_squared_error(test_data['CrossFreqSIHCRatio'], y_preds_test))
    
    ridge_results.append({
        'alpha': alpha,
        'RMSE_train': RMSE_train,
        'RMSE_test': RMSE_test,
        'diff': RMSE_train - RMSE_test
    })

ridge_results

[{'alpha': 0.001,
  'RMSE_train': 2.8455834814304968,
  'RMSE_test': 2.852246833174837,
  'diff': -0.006663351744340407},
 {'alpha': 0.01,
  'RMSE_train': 2.8498024244913065,
  'RMSE_test': 2.8555310747296168,
  'diff': -0.005728650238310262},
 {'alpha': 0.1,
  'RMSE_train': 3.178524520793153,
  'RMSE_test': 3.1793500112421538,
  'diff': -0.0008254904490008208},
 {'alpha': 1.0,
  'RMSE_train': 8.127294857570236,
  'RMSE_test': 8.130828026361188,
  'diff': -0.0035331687909518905}]

In [245]:
X_s = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz','Amplitude','Strain (x5)']]
y_s = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']
stratified_grouped_kfold_by_feature_ridge(X = X_s, y= y_s, data = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols, stratify_col = 'Strain (x5)', group_col='Subject', n_splits=5, numerical_cols=numerical_cols)

Fold 1/5: Train RMSE = 2.8208, Test RMSE = 2.9598
              Train groups: 84, Test groups: 21
              Train data points: 9629, Test data points: 2558
Fold 2/5: Train RMSE = 2.9051, Test RMSE = 2.7002
              Train groups: 84, Test groups: 21
              Train data points: 9614, Test data points: 2573
Fold 3/5: Train RMSE = 2.9207, Test RMSE = 2.5550
              Train groups: 84, Test groups: 21
              Train data points: 9795, Test data points: 2392
Fold 4/5: Train RMSE = 2.7828, Test RMSE = 3.1262
              Train groups: 84, Test groups: 21
              Train data points: 9913, Test data points: 2274
Fold 5/5: Train RMSE = 2.8111, Test RMSE = 3.0188
              Train groups: 84, Test groups: 21
              Train data points: 9797, Test data points: 2390

Final Results - Avg Train RMSE = 2.84809, Avg Test RMSE = 2.87200


(2.8480949151079744, 2.8719961215263488)

In [251]:
X_g = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols[['FreqkHz','Amplitude', 'Group']]
y_g = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols['CrossFreqSIHCRatio']
stratified_grouped_kfold_by_feature_ridge(X = X_g, y= y_g, data = final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols, stratify_col = 'Group', group_col='Subject', n_splits=5, numerical_cols=numerical_cols)

Fold 1/5: Train RMSE = 2.8013, Test RMSE = 3.0530
              Train groups: 84, Test groups: 21
              Train data points: 9883, Test data points: 2304
Fold 2/5: Train RMSE = 2.8133, Test RMSE = 2.9886
              Train groups: 84, Test groups: 21
              Train data points: 9856, Test data points: 2331
Fold 3/5: Train RMSE = 2.7722, Test RMSE = 3.1319
              Train groups: 84, Test groups: 21
              Train data points: 9665, Test data points: 2522
Fold 4/5: Train RMSE = 2.9242, Test RMSE = 2.6332
              Train groups: 84, Test groups: 21
              Train data points: 9475, Test data points: 2712
Fold 5/5: Train RMSE = 2.9339, Test RMSE = 2.4790
              Train groups: 84, Test groups: 21
              Train data points: 9869, Test data points: 2318

Final Results - Avg Train RMSE = 2.84898, Avg Test RMSE = 2.85715


(2.8489799511738716, 2.8571476500446336)

In [152]:
model_ols = smf.ols('CrossFreqSIHCRatio ~ FreqkHz + LeveldB + Amplitude + FreqkHz*Amplitude + FreqkHz*Amplitude*LeveldB', data=final_clean_strained_grouped_pos_cleangroup_vs_timed2_avgvx_encoded_ols).fit()
print(model_ols.summary())

                            OLS Regression Results                            
Dep. Variable:     CrossFreqSIHCRatio   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     234.2
Date:                Tue, 01 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:22:32   Log-Likelihood:                -29556.
No. Observations:               12187   AIC:                         5.913e+04
Df Residuals:                   12179   BIC:                         5.919e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             